## 6. Entrene una red convolucional para clasificar las imágenes de la base de datos MNIST. ¿Cuál es la red convolucional más pequeña que puede conseguir con una exactitud de al menos 90% en el conjunto de evaluación? ¿Cuál es el perceptrón multicapa más pequeño que puede conseguir con la misma exactitud?

Podemos empezar con una red como la del ejemplo hasta encontrar la mas chica (a nivel parámetros). Lograr el desempeño y 
ver que puedo sacarle sin que el desempeño caiga (a nivel parámetros)

### Definicion de la red

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [22]:
# Punto de entrada brindada por la cátedra.
# La clase hereda de Module
class conv(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0) # capa convolucional (usa filtros no neuronas)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) 
        self.conv2 = nn.Conv2d(6, 16, 5) # 2da capa convolucional (usa filtros no neuronas)
        self.fc1 = nn.Linear(400, 120) # primera capa. 120 neuronas
        self.fc2 = nn.Linear(120, 84)  # capa intermedia . 84 neuronas
        self.fc3 = nn.Linear(84, 10)   # capa de salida
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [23]:
# Se cargan a memoria los tensores de train/test (imágenes y etiquetas) desde archivos .pt
# Me devuelve en cada caso un tensor -> al tensor lo puedo pensar arreglo (array) n-dimensional:

# 0-D: escalar (un número)
# 1-D: vector (ej. shape [10])
# 2-D: matriz (ej. [32, 10])
# 3-D+: tensores con más ejes (imágenes, videos, batches, etc.)

#Imágenes → features que entran a la red.
#Labels → objetivo/“respuesta correcta” para calcular la pérdida y guiar el aprendizaje.

device = 'cpu'
training_data = torch.load('MNIST_training_data.pt').to(torch.float32).to(device)
training_labels = torch.load('MNIST_training_labels.pt').to(device)
test_data = torch.load('MNIST_test_data.pt').to(torch.float32).to(device)
test_labels = torch.load('MNIST_test_labels.pt').to(device)

In [24]:
# Esto me devuelve: torch.Size([60000, 28, 28])
# 60000 → cantidad de imágenes (todo el set de entrenamiento de MNIST).
#28, 28 → alto y ancho de cada imagen (en grises).

training_data.size()

torch.Size([60000, 28, 28])

In [25]:
# Asegurar tipos/shape antes (una sola vez):
training_labels = training_labels.long()
test_labels     = test_labels.long()
# Si tus imágenes están [N,28,28], agregá canal:
if training_data.ndim == 3:
    training_data = training_data.unsqueeze(1)
    test_data     = test_data.unsqueeze(1)

In [26]:
f_loss = nn.CrossEntropyLoss() # función de perdida
modelo = conv().to(device)
optimizer = torch.optim.SGD(modelo.parameters(), lr=0.001)

In [27]:
def make_minibatches(x, y, batch_size: int, shuffle: bool = True):
    """
    Devuelve un DataLoader que entrega (xb, yb) ya en mini-batches.
    x: Tensor de imágenes con shape [N, C, H, W]
    y: Tensor de labels con shape [N]
    """
    ds = TensorDataset(x, y)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

In [28]:
# La idea es entrenar en mini-batches a lo largo de varias "epocas"
# La epoca (epoch) es una pasada completa por todo el dataset de entrenamiento.
N_epocas = 10
minibatch_size = 128

for epoca in range(N_epocas):
    modelo.train()
    train_loader = make_minibatches(training_data, training_labels, minibatch_size, shuffle=True)

    total_loss, total_ej = 0.0, 0
    for xb, yb in train_loader:              # <-- acá viene cada mini-batch
        xb = xb.to(device)
        yb = yb.long().to(device)            # CrossEntropy: labels en long

        outputs = modelo(xb)
        loss = f_loss(outputs, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * xb.size(0)
        total_ej   += xb.size(0)

    print(f"Época {epoca+1}: loss_prom={total_loss/total_ej:.4f}")

RuntimeError: Given groups=1, weight of size [6, 3, 5, 5], expected input[128, 1, 28, 28] to have 3 channels, but got 1 channels instead